In [1]:
!pip install azureml-sdk

In [2]:
import os
import json
import requests

from azureml.core import Workspace
from azureml.core import Model
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice, Webservice

C:\Users\HP\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [3]:
try:
    os.rename('env','.env')
    print("File renamed to .env successfully")
except FileNotFoundError:
    print('The file env was not found!')
except Exception as e:
    print(f"An error occurred: {e}")

File renamed to .env successfully


In [4]:
print(os.listdir())

['.env', '.ipynb_checkpoints', '1. Diabetes model to be deployed.ipynb', 'Deploy ML model on Azure.ipynb', 'diabetes.csv', 'diabetes_model.pkl']


In [6]:
from dotenv import load_dotenv

In [12]:
load_dotenv(".env")

subscription_id=os.getenv('SUBSCRIPTION_ID')
resource_group=os.getenv('RESOURCE_GROUP')
workspace_name=os.getenv('WORKSPACE_NAME')
region=os.getenv('REGION')

print(resource_group,workspace_name,region)

ml-model-deployment-rg deploy-sklearn-model-ws centralindia


In [14]:
#create workspace

ws=Workspace.create(name=workspace_name,
                   subscription_id=subscription_id,
                   resource_group=resource_group,
                   location=region)

print(f'Workspace {workspace_name} created')

Performing interactive authentication. Please follow the instructions on the terminal.


The default web browser has been opened at https://login.microsoftonline.com/organizations/oauth2/v2.0/authorize. Please continue the login in the web browser. If no web browser is available or if the web browser fails to open, use device code flow with `az login --use-device-code`.


Interactive authentication successfully completed.
Deploying StorageAccount with name deployskstorage5d070bb26.
Deploying KeyVault with name deployskkeyvault79a5375c.
Deployed KeyVault with name deployskkeyvault79a5375c. Took 100.59 seconds.
Deploying AppInsights with name deployskinsights1bdff483.
Deployed AppInsights with name deployskinsights1bdff483. Took 3.06 seconds.
Deployed StorageAccount with name deployskstorage5d070bb26. Took 111.92 seconds.
Deploying Workspace with name deploy-sklearn-model-ws.
Deployed Workspace with name deploy-sklearn-model-ws. Took 41.32 seconds.
Workspace deploy-sklearn-model-ws created


In [15]:
model_path='diabetes_model.pkl'
model_name='diabetes_prediction_model'

In [16]:
registered_model=Model.register(model_path=model_path,model_name=model_name,workspace=ws)

Registering model diabetes_prediction_model


In [17]:
conda_env=Environment('my-conda-env')
conda_env.python.conda_dependencies=CondaDependencies.create(conda_packages=['scikit-learn'])

In [18]:
inference_config=InferenceConfig(entry_script='score.py',environment=conda_env)

In [19]:
aci_config=AciWebservice.deploy_configuration(cpu_cores=1,memory_gb=1)

In [22]:
service=Model.deploy(workspace=ws,
                    name='diabetes-prediction-service',
                    models=[registered_model],
                    inference_config=inference_config,
                    deployment_config=aci_config)
service.wait_for_deployment(show_output=True)

C:\Users\HP\AppData\Local\Temp\ipykernel_16892\331850269.py:1: FutureWarning: azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'
  service=Model.deploy(workspace=ws,


Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2024-12-24 16:06:57+05:30 Creating Container Registry if not exists..
2024-12-24 16:16:57+05:30 Registering the environment.
2024-12-24 16:17:01+05:30 Building image..
2024-12-24 16:27:16+05:30 Generating deployment configuration.
2024-12-24 16:27:18+05:30 Submitting deployment to compute..
2024-12-24 16:27:28+05:30 Checking the status of deployment diabetes-prediction-service..
2024-12-24 16:28:46+05:30 Checking the status of inference endpoint diabetes-prediction-service.
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [23]:
scoring_uri=service.scoring_uri

In [24]:
scoring_uri

'http://e21ef61a-e945-4201-88f1-e045dfd6215e.centralindia.azurecontainer.io/score'

# Cleaning up all the resources

In [26]:
#delete service

service_name='diabetes-prediction-service'
service=ws.webservices[service_name]
service.delete()

Running
2024-12-24 17:02:45+05:30 Check and wait for operation (e30f42f1-2daf-4ca7-ba66-cfa4c0724c08) to finish.
2024-12-24 17:02:49+05:30 Deleting service entity.
Succeeded


In [27]:
#name of the model to be deleted
model_name='diabetes_prediction_model'

#delete the model
registered_model=ws.models[model_name]
registered_model.delete()

In [28]:
#delete workspace

ws.delete(delete_dependent_resources=True)